In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
df = pd.read_csv('combined_with_split_tags.csv')
df

,Tag,Cleaned_Text,Length,Emoticons_count,Emoticons Avg,Unique_Words,TTR,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Afinn Score,Polarity,Subjectivity,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject,lowercase,uppercase,uppercase_num,proper cap,contractions_num,emotionalpunctuations_num,readable_num,hedge_perc,firstperson_perc,thirdperson_perc,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,characters_per_word,syll_per_word,words_per_sentence,sentences_per_paragraph,type_token_ratio,characters,syllables,words,wordtypes,sentences,paragraphs,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,pronoun,preposition,nominalization,pronoun.1,interrogative,article,subordination,conjunction.1,preposition.1,count,E_I,N_S,F_T,J_P
0,ENFP,disagree think theory given validity rather be...,1682,0,0.000000,25,0.014863,0.002378,0.003567,0.000595,0.005945,0.002378,0.007134,0.013674,0.002973,0.000595,0.008918,3,0.059524,0.436508,124,42,61,53,23,85,34,0,0,0,15,1,0.00,3,486,0.438272,0.014403,0.024691,32.261518,39.660000,11.086820,3.580430,37.599575,101.642251,22.104973,18.166667,12.927531,4.636943,1.460722,78.500000,6.0,0.515924,2184,688,471,243,6,1,109,73,161,23,6,25,36,67,23,3,0,1,0,0,0,228,E,N,F,P
1,INTP,selfish generally perform task better person a...,1265,0,0.000000,26,0.020553,0.010277,0.006324,0.010277,0.001581,0.002372,0.012648,0.011858,0.003953,0.000791,0.005534,5,-0.009722,0.660648,73,25,46,30,53,92,27,0,0,0,25,1,0.02,1,401,0.443890,0.089776,0.039900,38.985804,48.357195,8.449873,-5.890321,44.289526,118.205112,20.748239,18.000000,12.389050,4.174564,1.311721,100.250000,4.0,0.436409,1674,526,401,175,4,1,72,42,96,18,2,18,60,51,7,3,0,0,0,0,0,177,I,N,T,P
2,INFP,personality system technically separate types...,738,0,0.000000,25,0.033875,0.001355,0.000000,0.004065,0.001355,0.000000,0.001355,0.006775,0.000000,0.000000,0.002710,12,0.121053,0.382310,55,26,31,33,16,34,18,0,0,0,4,1,0.02,0,269,0.498141,0.040892,0.048327,102.699182,131.734907,7.390028,-162.121933,110.871375,280.524164,28.690465,31.000000,21.792209,3.962825,1.133829,269.000000,1.0,0.531599,1066,305,269,143,1,1,31,22,82,16,3,16,21,28,3,0,0,1,0,0,0,129,I,N,F,P
3,INTP,like functions general theyre basically ways p...,1550,0,0.000000,25,0.016129,0.002581,0.006452,0.002581,0.003871,0.007097,0.005806,0.018065,0.003871,0.003871,0.014194,10,0.108208,0.371320,80,35,55,33,33,82,25,0,0,0,3,1,0.01,2,389,0.519280,0.056555,0.051414,18.923333,22.510195,11.457340,36.177222,23.945475,69.142039,17.719601,11.222222,11.065275,4.752577,1.500000,43.111111,9.0,0.536082,1844,582,388,208,9,1,101,65,130,11,2,24,35,51,10,0,1,0,0,0,0,206,I,N,T,P
4,ENTP,even referring nihilism yeah point dying livin...,767,0,0.000000,25,0.032595,0.011734,0.007823,0.006519,0.019557,0.005215,0.026076,0.013038,0.019557,0.002608,0.006519,-31,0.006090,0.518506,42,17,25,17,19,47,26,0,0,0,2,1,0.04,3,231,0.445887,0.004329,0.047619,90.284416,114.418831,9.474105,-140.796234,96.382684,249.181818,29.267851,42.000000,19.058689,4.320346,1.337662,231.000000,1.0,0.632035,998,309,231,146,1,1,42,23,58,13,6,12,21,30,3,1,0,0,0,0,0,110,E,N,T,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,ISTP,kinda amusing deal trust relate usually someon...,599,0,0.000000,24,0.040067,0.000000,0.005008,0.001669,0.000000,0.005008,0.003339,0.006678,0.003339,0.001669,0.005008,6,0.150000,0.594444,46,13,10,11,26,51,18,0,0,0,9,0,0.06,10,196,0.515306,0.137755,0.035714,74.877551,94.352653,6.247678,-92.675408,81.053061,207.224490,22.748418,22.000000,17.708406,3.775510,1.188776,196.000000,1.0,0.668367,740,233,196,131,1,1,22,13,54,4,3,7,36,20,0,0,0,0,0,0,0,88,I,S,T,P
12822,ESFP,social 9

In [3]:
df['FT'] = df['Tag'].apply(lambda x: 1 if 'F' in x else 0)
df['FT']

0        1
1        0
2        1
3        0
4        0
        ..
12821    0
12822    1
12823    1
12824    1
12825    1
Name: FT, Length: 12826, dtype: int64

In [4]:
# # Parameters for RandomizedSearchCV
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}


# Another set of paramters, if you want to try
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200, 300],
    'class_weight': ['balanced', 'balanced_subsample']
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)


In [5]:
FT_allfeatures = df.drop(columns = ['Tag', 'FT','E_I',	'N_S',	'F_T',	'J_P'])
FT_y = df['FT']

X_train, X_test, y_train, y_test = train_test_split(FT_allfeatures, FT_y, test_size=0.2, random_state=1)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [6]:
X_train_cleanedtext = X_train['Cleaned_Text']
X_train_features = X_train.drop(columns = ['Cleaned_Text'])
X_train_features = X_train_features.reset_index(drop = True)

X_test_cleanedtext = X_test['Cleaned_Text']
X_test_features = X_test.drop(columns = ['Cleaned_Text'])
X_test_features = X_test_features.reset_index(drop = True)

In [7]:
# Word Representation

vect = CountVectorizer(max_features = 5000)
X_train_dtm = vect.fit_transform(X_train_cleanedtext)
X_test_dtm = vect.transform(X_test_cleanedtext)

In [8]:
X_train_dtm_df = pd.DataFrame(X_train_dtm.toarray(), columns = vect.get_feature_names())
X_test_dtm_df = pd.DataFrame(X_test_dtm.toarray(), columns = vect.get_feature_names())

X_train_allfeatures = pd.concat([X_train_dtm_df, X_train_features], axis=1)
X_test_allfeatures = pd.concat([X_test_dtm_df, X_test_features], axis=1)

In [9]:
# X_train_allfeatures.isna()
# X_train_allfeatures[X_train_allfeatures.isna().any(axis=1)]

In [10]:
grid_search.fit(X_train_allfeatures,y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 49.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [11]:
print("Best parameters set found on development set:")
print()
print(grid_search.best_params_)
print()

Best parameters set found on development set:

{'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 80, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}



In [12]:
# Predicting on the transformed data
grid_predictions = grid_search.predict(X_test_allfeatures)

# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, grid_predictions )

0.6769290724863601

In [16]:
import pickle
pickle.dump(grid_search, open("model_ft.sav", "wb"))
